In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import xlrd
import matplotlib.pyplot as plt
import re, collections
from sklearn.ensemble import RandomForestRegressor
from itertools import chain

In [3]:
#read in training data
df = pd.read_excel("training_set_rel3.xls")

FileNotFoundError: [Errno 2] No such file or directory: 'training_set_rel3.xls'